In [9]:
import os
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

import mlflow
import mlflow.sklearn


def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2



if __name__ == "__main__":
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file (make sure you're running this from the root of MLflow!)
    wine_path = "df_final.csv"
    data = pd.read_csv(wine_path, sep=';', encoding='iso-8859-1')

    
    train, test = train_test_split(data, test_size=0.19)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["NOTAS"], axis=1)
    test_x = test.drop(["NOTAS"], axis=1)
    train_y = train[["NOTAS"]]
    test_y = test[["NOTAS"]]

    #alpha = 1
    #l1_ratio = 1

    with mlflow.start_run():
        lr = LinearRegression()
        lr.fit(train_x, train_y)

        predicted_qualities = lr.predict(test_x)

        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        print("LinearRegression model :")
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        mlflow.sklearn.log_model(lr, "model")

LinearRegression model :
  RMSE: 38.17171345229174
  MAE: 16.42947148669303
  R2: 0.8610511959463023
